# Implementation Karten

In [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/index.html) können Bereiche benutzerdefinierbar eingefärbt werden. Die Bereiche können im .geojson-Format bereit gestellt werden.

Wir benötigen die Imports `Map` und `GeoJSON` aus `ipyleaflet` und `json`, um die .geojson-Datei einzulesen.

In [1]:
from ipyleaflet import Map, GeoJSON
import json

Anschließend definieren wir ein dict mit den IDs der Bundesländer als Schlüssel, und den zugehörigen Farben als Wert.

In [2]:
colors = {
    "DE-BW": "#008578",    # Baden-Württemberg
    "DE-BY": "#129FD4",    # Bayern
    "DE-BE": "#753BBD",    # Berlin
    "DE-BB": "#4CA72B",    # Brandenburg
    "DE-HB": "#0663B5",    # Bremen
    "DE-HH": "#BD0F77",    # Hamburg
    "DE-HE": "#0663B5",    # Hessen
    "DE-MV": "#0663B5",    # Mecklenburg-Vorpommern
    "DE-NI": "#DE2817",    # Niedersachsen
    "DE-NW": "#129FD4",    # Nordrhein-Westfalen
    "DE-RP": "#CD8402",    # Rheinland-Pfalz
    "DE-SL": "#DE2817",    # Saarland
    "DE-SN": "#757575",    # Sachsen
    "DE-ST": "#BD0F77",    # Sachsen-Anhalt
    "DE-SH": "#4CA72B",    # Schleswig-Holstein
    "DE-TH": "#CD8402",    # Thüringen
}

Jetzt lesen wir eine [.geojson-Datei](https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/refs/heads/main/2_bundeslaender/2_hoch.geo.json) ein, die die Bundesländer Deutschlands enthält.

In [3]:
with open('germany_states.geojson', 'r') as f:
    data = json.load(f)

Die Farben werden den zugehörigen Bundesländern zugewiesen.

In [4]:
for feature in data['features']:
    state_id = feature['properties']['id']
    if state_id in colors:
        feature['properties']['style'] = {
            'color': colors[state_id],
        }

Wir erstellen die `Map` zentriert auf Deutschland.

In [ ]:
m = Map(center=(51.1657, 10.4515), zoom=5)

Wir erstellen den `GeoJSON`-Layer und befüllen ihn mit den Daten aus der .geojson-Datei. Die Angabe `weight` im `style`-Argument bestimmt die Strichstärke der Umrandung.  
Um die Karte anzuzeigen geben wir den Namen an.

In [6]:
geo_json = GeoJSON(
    data = data,
    style = {
       'weight': .1
    }
)
m.add(geo_json)
m

Map(center=[51.1657, 10.4515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…